## Install Tensorflow 1.15

In [ ]:
! apt-get install libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip libjpeg8-dev liblapack-dev libblas-dev gfortran
! pip3 install -U numpy==1.16.1 future==0.18.2 mock==3.0.5 h5py==2.10.0 keras_preprocessing==1.1.1 keras_applications==1.0.8 gast==0.2.2 futures protobuf pybind11
! sudo pip3 install --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v45 'tensorflow<2'

## Install TensorRT Runtime

In [ ]:
%%bash
wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

dpkg -i nvidia-machine-learning-repo-*.deb
apt-get update

sudo apt-get install libnvinfer5

## Clone TF-TRT Models

In [ ]:
%cd /content/
%rm -rf tf_trt_models
!git clone --recursive https://github.com/NVIDIA-Jetson/tf_trt_models.git
%cd tf_trt_models

## Compile TF-TRT Model

In [ ]:
!./install.sh python3

## Download Pretrained Model Tensorflow (TF Model Zoo - TF1)
- [TF Model Zoo TF1](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md)

In [ ]:

%rm -rf /content/tf_models
%mkdir /content/tf_models
%cd /content/tf_models

import os
import shutil
import glob
import urllib.request
import tarfile


MODELS = ['ssd_inception_v2_coco_2018_01_28',
          'ssd_mobilenet_v1_coco_2018_01_28']

DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

for MODEL in MODELS:
  MODEL_FILE = MODEL + '.tar.gz'

  with urllib.request.urlopen(DOWNLOAD_BASE+MODEL_FILE) as response, open(MODEL_FILE, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

  tar = tarfile.open(MODEL_FILE)
  tar.extractall()
  tar.close()

  os.remove(MODEL_FILE)

## Install Tensorflow Object Detection API

In [ ]:
! pip3 uninstall tensorflow-object-detection-api -y
! pip3 install tensorflow-object-detection-api

## Build Inference Graph 

In [ ]:
%cd /content/tf_trt_models/

from tf_trt_models.detection import build_detection_graph

MODEL_NAME = "ssd_mobilenet_v1_coco_2018_01_28"
config_path = "/content/tf_models/%s/pipeline.config" % MODEL_NAME
checkpoint_path = "/content/tf_models/%s/model.ckpt" % MODEL_NAME

frozen_graph, input_names, output_names = build_detection_graph(
    config=config_path,
    checkpoint=checkpoint_path,
    score_threshold=0.3,
    batch_size=1
)

## Crete TF-TRT Graph 

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50  
)

## Save TF-TRT Graph

In [ ]:
%mkdir -p /content/tf-trt-graph/

with open('/content/tf-trt-graph/%s_trt.pb' % MODEL_NAME, 'wb') as f:
    f.write(trt_graph.SerializeToString())

## Load TF-TRT Graph

In [ ]:
import tensorflow as tf 

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.Session(config=tf_config)

tf.import_graph_def(frozen_graph, name='')

tf_input = tf_sess.graph.get_tensor_by_name(input_names[0] + ':0')
tf_scores = tf_sess.graph.get_tensor_by_name('detection_scores:0')
tf_boxes = tf_sess.graph.get_tensor_by_name('detection_boxes:0')
tf_classes = tf_sess.graph.get_tensor_by_name('detection_classes:0')
tf_num_detections = tf_sess.graph.get_tensor_by_name('num_detections:0')

## Load Image

In [ ]:
%cd /content/
!gdown --id 1v5A4RIBYxjump3AJ8eoqpTRry4H-VAq6 # download "zebra.jpg" from google drive

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline

image = Image.open("zebra.jpg")

image_resized = np.array(image.resize((300, 300)))
image = np.array(image)

plt.imshow(image)

## Load Class Map data (coco.json)

In [ ]:
% cd /content/
!gdown --id 1XyPDMZh-vZEr48QiWJ6ANouSnyxqPRXN

In [ ]:
import json 
with open("coco.json", "r") as f:
  name_classes = json.load(f)

## Detect Object on Image 

In [ ]:
scores, boxes, classes, num_detections = tf_sess.run([tf_scores, 
                                                      tf_boxes, 
                                                      tf_classes, 
                                                      tf_num_detections], 
                                                     feed_dict={
                                                          tf_input: image_resized[None, ...]
                                                      })

boxes = boxes[0] # index by 0 to remove batch dimension
scores = scores[0]
classes = classes[0]
num_detections = num_detections[0]

## Display Result

In [ ]:
import matplotlib.patches as patches

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.imshow(image)

# plot boxes exceeding score threshold
for i in range(int(num_detections)):
    # scale box to image coordinates
    box = boxes[i] * np.array([image.shape[0], image.shape[1], image.shape[0], image.shape[1]])

    # display rectangle
    patch = patches.Rectangle((box[1], box[0]), box[3] - box[1], box[2] - box[0], color='g', alpha=0.3)
    ax.add_patch(patch)

    # display class index and score
    name = name_classes[str(int(classes[i]))]
    plt.text(x=box[1] + 10, y=box[2] - 10, s='%s (%0.2f) ' % (name, scores[i]), color='w')

plt.show()

## Measure Average Runtime TF-TRT Graph

In [ ]:
import time 

num_samples = 50

t0 = time.time()
for i in range(num_samples):
    scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={
        tf_input: image_resized[None, ...]
    })
t1 = time.time()
print('Average runtime: %f seconds' % (float(t1 - t0) / num_samples))

## Close Session

In [ ]:
tf_sess.close()

## Zip & Download TF-TRT Graph

In [ ]:
%cd /content/
!zip -r tf_trt_graph.zip /content/tf-trt-graph
    
from google.colab import files
files.download("tf_trt_graph.zip")